#### Cria repositório de reports

In [1]:
from utils.enums import Datas
from utils.checkpoints import verifyPath

teste_size = 2
main_data = Datas.MOTION
path_reports = f"report_results/{Datas.HAR.value}/{main_data.value}_{teste_size}/"

split_path = path_reports.split("/")
partial_path = ""
for i, part in enumerate(split_path):
    partial_path += part + "/"
    verifyPath(partial_path)


# Treina Pretexto

#### Hiperparâmentos

In [2]:
import math
import torch
import numpy as np
import pandas as pd
from models.cnn1d import CNN1d
from utils.enums import Datas, Sets, ModelTypes
from data_modules.pretext import HarDataModule as HarDataModulePretext
from transforms.har import rotation, flip, noise_addition, permutation, scaling, time_warp, negation

printQtd = 1
num_epoch = 320
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optim
learning_rate = 0.02
step_size = 80
gamma = 0.5

#### Carrega Dados

In [3]:
transforms = [rotation, flip, noise_addition, permutation, scaling, time_warp, negation]
data_module = HarDataModulePretext(batch_size=batch_size, main_data = main_data)
train_dl, train_ds = data_module.get_dataloader(set=Sets.TRAIN.value, shuffle=True, transforms=transforms)
test_dl, test_ds   = data_module.get_dataloader(set=Sets.TEST.value, shuffle=False, transforms=transforms)
num_classes = len(transforms)

#### Define Modelo

In [4]:
model_p = CNN1d(data_label=main_data.value, num_classes=num_classes, require_grad=True, type=ModelTypes.PRETEXT.value)
print(model_p)

CNN1d(
  (criterion): CrossEntropyLoss()
  (backbone): Backbone(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv1d(6, 12, kernel_size=(2,), stride=(1,))
    (conv2): Conv1d(12, 24, kernel_size=(2,), stride=(1,))
    (conv3): Conv1d(24, 48, kernel_size=(2,), stride=(1,))
  )
  (pred_head): ProjectionHead(
    (linear1): Linear(in_features=288, out_features=7, bias=True)
  )
)


#### Treinamento

In [5]:
# import lightning as L
# trainer = L.Trainer(
#     max_epochs=10,
#     accelerator='cpu',
#     log_every_n_steps=1        
# )
# trainer.fit(model=model, train_dataloaders=train_dl)

optimizer, lr_scheduler = model_p.configure_optimizers(step_size=step_size, gamma=gamma, learning_rate=learning_rate)
optimizer, lr_scheduler = optimizer[0], lr_scheduler[0]

train_errors = []
validation_errors = []
best_val_loss = 500
n_total_steps = len(train_dl)
for epoch in range(num_epoch):
    # Treinamento
    model_p.train()
    train_loss = 0
    for i, batch in enumerate(train_dl):
        loss = model_p.training_step(batch)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % math.floor(n_total_steps/printQtd) == 0:
            print (f'Epoch [{epoch+1:4d}/{num_epoch}], Step [{i+1:4d}/{n_total_steps}], Loss: {loss.item():.4f}', end= "" if n_total_steps/printQtd+i >= n_total_steps else "\n")

    lr_scheduler.step()

    # Validação
    model_p.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dl:
            val_loss += model_p.validation_step(batch)
    
    val_loss /= len(test_dl)
    print(f' Validation Loss: {val_loss:.4f}')

    train_errors.append(train_loss/len(train_dl))
    validation_errors.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model_p.state_dict()  # Salva os parâmetros do modelo

model_p.load_state_dict(best_model)

/home/sr_rosa/.local/lib/python3.10/site-packages/lightning/pytorch/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


Epoch [   1/320], Step [ 104/104], Loss: 1.9414 Validation Loss: 1.9451
Epoch [   2/320], Step [ 104/104], Loss: 1.9578 Validation Loss: 1.9464
Epoch [   3/320], Step [ 104/104], Loss: 1.9331 Validation Loss: 1.9434
Epoch [   4/320], Step [ 104/104], Loss: 1.9609 Validation Loss: 1.9438
Epoch [   5/320], Step [ 104/104], Loss: 1.9477 Validation Loss: 1.9412
Epoch [   6/320], Step [ 104/104], Loss: 1.9508 Validation Loss: 1.9403
Epoch [   7/320], Step [ 104/104], Loss: 1.9353 Validation Loss: 1.9382
Epoch [   8/320], Step [ 104/104], Loss: 1.9671 Validation Loss: 1.9408
Epoch [   9/320], Step [ 104/104], Loss: 1.9364 Validation Loss: 1.9338
Epoch [  10/320], Step [ 104/104], Loss: 1.9212 Validation Loss: 1.9293
Epoch [  11/320], Step [ 104/104], Loss: 1.9571 Validation Loss: 1.9356
Epoch [  12/320], Step [ 104/104], Loss: 1.9127 Validation Loss: 1.9311
Epoch [  13/320], Step [ 104/104], Loss: 1.9036 Validation Loss: 1.9250
Epoch [  14/320], Step [ 104/104], Loss: 1.9047 Validation Loss:

<All keys matched successfully>

#### Avaliação

In [6]:
accTotal = 0
predicted_values = []
real_values = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    n_each_class_samples = [0 for i in range(num_classes)]

    for data, labels in test_dl:
        outputs = model_p(data)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for pred, real in zip (predicted, labels):
            predicted_values.append(pred.item())
            real_values.append(real.item())

        for i in range(labels.shape[0]):
            label = labels[i]
            pred  = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            n_each_class_samples[pred] += 1

    accTotal = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accTotal} %')

    for i in range(num_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {test_ds.getLabel(i)} ({n_class_correct[i]}/{n_class_samples[i]} | {n_each_class_samples[i]}): {acc} %')

Accuracy of the network: 74.56078706957133 %
Accuracy of Rotation (137/206 | 153): 66.50485436893204 %
Accuracy of Flip (137/203 | 181): 67.48768472906404 %
Accuracy of Noise_addition (192/209 | 245): 91.86602870813397 %
Accuracy of Permutation (156/209 | 249): 74.64114832535886 %
Accuracy of Scaling (140/195 | 164): 71.7948717948718 %
Accuracy of Time Warp (115/203 | 145): 56.65024630541872 %
Accuracy of Negation (184/198 | 286): 92.92929292929293 %


#### Salva resultados

In [7]:
model_p.save_backbone(accuracy=accTotal, batch_size=batch_size, num_epoch=num_epoch)

pred_reports = pd.DataFrame({
    Sets.REAL.value: real_values,
    Sets.PREDICTION.value : predicted_values
})
pred_reports.to_csv(f"{path_reports}/predictions_{ModelTypes.PRETEXT.value}.dat", sep=" ", index=False)

train_reports = pd.DataFrame({
    Sets.TRAIN.value : train_errors,
    Sets.VALIDATION.value : validation_errors
})
train_reports.to_csv(f"{path_reports}/errors_{ModelTypes.PRETEXT.value}.dat", sep=" ", index=False)

# Treina Downstream

#### Hiperparâmentos

In [18]:
import torch
import math
import numpy as np
import pandas as pd
from data_modules.har import HarDataModule as HarDataModuleDownstram
from utils.enums import Datas, Sets, ModelTypes
from models.cnn1d import CNN1d

printQtd = 1
isFreezing = False
batch_size = 10
num_epoch = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optim config
learning_rate_bb = 0.02
step_size_bb = 40
gamma_bb = 0.5

learning_rate_ds = 0.02
step_size_ds = 20
gamma_ds = 0.5

#### Carrega Dados

In [19]:
data_module = HarDataModuleDownstram(batch_size=batch_size)
train_dl, train_ds = data_module.get_dataloader(set=Sets.TRAIN.value, shuffle=True)
test_dl, test_ds   = data_module.get_dataloader(set=Sets.TEST.value, shuffle=True)
num_classes = len(train_ds.labels)

#### Define Modelo

In [20]:
model = CNN1d(
    data_label=main_data.value, 
    num_classes=num_classes, 
    require_grad= not isFreezing, 
    type=ModelTypes.DOWNSTREAM.value
)
print(model)

path:  ./best_models/backbone_motion.pth
CNN1d(
  (criterion): CrossEntropyLoss()
  (backbone): Backbone(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv1d(6, 12, kernel_size=(2,), stride=(1,))
    (conv2): Conv1d(12, 24, kernel_size=(2,), stride=(1,))
    (conv3): Conv1d(24, 48, kernel_size=(2,), stride=(1,))
  )
  (pred_head): PredictionHead(
    (linear1): Linear(in_features=288, out_features=6, bias=True)
  )
)


#### Treinamento

In [21]:
# import lightning as L
# trainer = L.Trainer(
#     max_epochs=10,
#     accelerator='cpu',
#     log_every_n_steps=1        
# )
# trainer.fit(model=model, train_dataloaders=train_dl)

optimizer_backbone, lr_scheduler_backbone = model.configure_backbone_optimizers(step_size=step_size_bb, gamma=gamma_bb, learning_rate=learning_rate_bb)
optimizer_downstream, lr_scheduler_downstream = model.configure_head_optimizers(step_size=step_size_ds, gamma=gamma_ds, learning_rate=learning_rate_ds)

train_errors = []
validation_errors = []
best_val_loss = 500
n_total_steps = len(train_dl)
for epoch in range(num_epoch):
    # Treinamento
    model.train()
    train_loss = 0
    for i, batch in enumerate(train_dl):
        loss = model.training_step(batch)
        train_loss += loss.item()
        optimizer_backbone.zero_grad()
        optimizer_downstream.zero_grad()
        loss.backward()
        optimizer_backbone.step()
        optimizer_downstream.step()
        
        if (i+1) % math.floor(n_total_steps/printQtd) == 0:
            print (f'Epoch [{epoch+1:4d}/{num_epoch}], Step [{i+1:4d}/{n_total_steps}], Loss: {loss.item():.4f}', end= "" if n_total_steps/printQtd+i >= n_total_steps else "\n")

    lr_scheduler_backbone.step()
    lr_scheduler_downstream.step()

    # Validação
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dl:
            val_loss += model.validation_step(batch)
    
    val_loss /= len(test_dl)
    print(f' Validation Loss: {val_loss:.4f}')

    train_errors.append(train_loss/len(train_dl))
    validation_errors.append(val_loss.item())

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict()  # Salva os parâmetros do modelo

model.load_state_dict(best_model)

/home/sr_rosa/.local/lib/python3.10/site-packages/lightning/pytorch/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


Epoch [   1/200], Step [   6/6], Loss: 1.7063 Validation Loss: 1.8147
Epoch [   2/200], Step [   6/6], Loss: 1.8525 Validation Loss: 1.7750
Epoch [   3/200], Step [   6/6], Loss: 1.7964 Validation Loss: 1.7231
Epoch [   4/200], Step [   6/6], Loss: 1.7385 Validation Loss: 1.7411
Epoch [   5/200], Step [   6/6], Loss: 1.7045 Validation Loss: 1.7050
Epoch [   6/200], Step [   6/6], Loss: 1.6171 Validation Loss: 1.6825
Epoch [   7/200], Step [   6/6], Loss: 1.8348 Validation Loss: 1.6748
Epoch [   8/200], Step [   6/6], Loss: 1.7940 Validation Loss: 1.6772
Epoch [   9/200], Step [   6/6], Loss: 1.6334 Validation Loss: 1.7316
Epoch [  10/200], Step [   6/6], Loss: 1.6059 Validation Loss: 1.6074
Epoch [  11/200], Step [   6/6], Loss: 1.6381 Validation Loss: 1.6728
Epoch [  12/200], Step [   6/6], Loss: 1.6562 Validation Loss: 1.6016
Epoch [  13/200], Step [   6/6], Loss: 1.7904 Validation Loss: 1.7400
Epoch [  14/200], Step [   6/6], Loss: 1.5490 Validation Loss: 1.5375
Epoch [  15/200], St

<All keys matched successfully>

#### Avaliação

In [22]:
accTotal = 0
predicted_values = []
real_values = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]
    n_each_class_samples = [0 for i in range(num_classes)]

    for data, labels in test_dl:
        outputs = model(data)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for pred, real in zip (predicted, labels):
            predicted_values.append(pred.item())
            real_values.append(real.item())

        for i in range(labels.shape[0]):
            label = labels[i]
            pred  = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            n_each_class_samples[pred] += 1

    accTotal = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {accTotal} %')

    for i in range(num_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {test_ds.getLabel(i)} ({n_class_correct[i]}/{n_class_samples[i]} | {n_each_class_samples[i]}): {acc} %')

Accuracy of the network: 33.333333333333336 %
Accuracy of Run (4/4 | 15): 100.0 %
Accuracy of Sit (4/4 | 8): 100.0 %
Accuracy of Stair-down (0/4 | 0): 0.0 %
Accuracy of Stair-up (0/4 | 1): 0.0 %
Accuracy of Stand (0/4 | 0): 0.0 %
Accuracy of Walk (0/4 | 0): 0.0 %


#### Salva Resultados

In [23]:
model.save_full_model(accuracy=accTotal, batch_size=batch_size, num_epoch=num_epoch)

pred_reports = pd.DataFrame({
    Sets.REAL.value: real_values,
    Sets.PREDICTION.value : predicted_values
})
pred_reports.to_csv(f"{path_reports}/predictions_{ModelTypes.DOWNSTREAM.value}.dat", sep=" ", index=False)

train_reports = pd.DataFrame({
    Sets.TRAIN.value : train_errors,
    Sets.VALIDATION.value : validation_errors
})
train_reports.to_csv(f"{path_reports}/errors_{ModelTypes.DOWNSTREAM.value}.dat", sep=" ", index=False)

## Realiza treinamento com percentuais da base de dados.

#### Hiperparâmentos

In [24]:
import torch
import math
import numpy as np
import pandas as pd
from data_modules.har import HarDataModule as HarDataModuleDownstram
from utils.enums import Datas, Sets, ModelTypes
from models.cnn1d import CNN1d

printQtd = 1
isFreezing = False
batch_size = 10
num_epoch = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optim config
learning_rate = 0.02
step_size = 50
gamma = 0.5

#### Realiza Testes

In [25]:
percents = [1, 20, 40, 60, 80, 100]

for percent in percents:

    # Carrega a base de dados
    data_module = HarDataModuleDownstram(batch_size=batch_size)
    train_dl, train_ds = data_module.get_dataloader(set=Sets.TRAIN.value, shuffle=True, percent=percent/100)
    test_dl, test_ds   = data_module.get_dataloader(set=Sets.TEST.value, shuffle=False)
    num_classes = len(test_ds.labels)

    # Define os modelos
    model1 = CNN1d(
        data_label=main_data.value, 
        num_classes=num_classes, 
        require_grad= not isFreezing, 
        type=ModelTypes.DOWNSTREAM.value
    )
    model2 = CNN1d(
        data_label=main_data.value, 
        num_classes=num_classes, 
        require_grad=True, 
        type=ModelTypes.PRETEXT.value
    )

    # Define os otimizadores
    optimizer1, lr_scheduler1 = model1.configure_optimizers(step_size=step_size, gamma=gamma, learning_rate=learning_rate)
    optimizer2, lr_scheduler2 = model2.configure_optimizers(step_size=step_size, gamma=gamma, learning_rate=learning_rate)

    optimizer1, lr_scheduler1 = optimizer1[0], lr_scheduler1[0]
    optimizer2, lr_scheduler2 = optimizer2[0], lr_scheduler2[0]

    # Realiza o treinamento
    best_val_loss1 = 500
    best_val_loss2 = 500
    n_total_steps = len(train_dl)
    for epoch in range(num_epoch):
        
        # Treinamento 
        model1.train()
        model2.train()
        for i, batch in enumerate(train_dl):
            if len(batch) <= 0:
                break
            loss1 = model1.training_step(batch)
            loss2 = model2.training_step(batch)

            optimizer1.zero_grad()
            optimizer2.zero_grad()

            loss1.backward()
            loss2.backward()

            optimizer1.step()
            optimizer2.step()

        lr_scheduler1.step()
        lr_scheduler2.step()

        # Validação
        model1.eval()
        model2.eval()

        val_loss1 = 0
        val_loss2 = 0

        with torch.no_grad():
            for batch in test_dl:
                val_loss1 += model1.validation_step(batch)
                val_loss2 += model2.validation_step(batch)
        
        val_loss1 /= len(test_dl)
        val_loss2 /= len(test_dl)

        if val_loss1 < best_val_loss1:
            best_val_loss1 = val_loss1
            best_model1 = model1.state_dict() 

        if val_loss2 < best_val_loss2:
            best_val_loss2 = val_loss2
            best_model2 = model2.state_dict()  

    model1.load_state_dict(best_model1)
    model2.load_state_dict(best_model2)

    predicted_values1 = []
    predicted_values2 = []

    real_values1 = []
    real_values2 = []
    
    with torch.no_grad():
        for data, labels in test_dl:
            outputs1 = model1(data)
            _, predicted1 = torch.max(outputs1, 1)

            for pred, real in zip (predicted1, labels):
                predicted_values1.append(pred.item())
                real_values1.append(real.item())

            outputs2 = model2(data)
            _, predicted2 = torch.max(outputs2, 1)

            for pred, real in zip (predicted2, labels):
                predicted_values2.append(pred.item())
                real_values2.append(real.item())

    pred_reports1 = pd.DataFrame({
        Sets.REAL.value: real_values1,
        Sets.PREDICTION.value : predicted_values1
    })
    pred_reports2 = pd.DataFrame({
        Sets.REAL.value: real_values2,
        Sets.PREDICTION.value : predicted_values2
    })

    pred_reports1.to_csv(f"report_results/har/percent/cnn/predictions_{percent}_m1.dat", sep=" ", index=False)
    pred_reports2.to_csv(f"report_results/har/percent/cnn/predictions_{percent}_m2.dat", sep=" ", index=False)

path:  ./best_models/backbone_motion.pth


/home/sr_rosa/.local/lib/python3.10/site-packages/lightning/pytorch/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


KeyboardInterrupt: 